In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
# Load the dataset

In [2]:
df = pd.read_csv('Depression.csv')
df.head()

,label,text,Date
0,0,Like 6 of my online friends have fallen foul o...,2019-05-31 16:32:55.447226
1,0,"@timeandtempest Y'know fibromyalgia's real, ri...",2021-01-24 16:32:55.447226
2,4,Jammin at the campfire what a jammin day!! ht...,2020-12-07 16:32:55.447226
3,0,"What's wrong today, it's been raining since 3p...",2018-08-28 16:32:55.447226
4,4,@Joy_Inc Lincoln - on Brayford Pool I can't c...,2020-09-26 16:32:55.447226


In [3]:
df['Date'] = df['Date'].apply(lambda x: x.split(' ')[0])
df.head()

,label,text,Date
0,0,Like 6 of my online friends have fallen foul o...,2019-05-31
1,0,"@timeandtempest Y'know fibromyalgia's real, ri...",2021-01-24
2,4,Jammin at the campfire what a jammin day!! ht...,2020-12-07
3,0,"What's wrong today, it's been raining since 3p...",2018-08-28
4,4,@Joy_Inc Lincoln - on Brayford Pool I can't c...,2020-09-26


In [4]:
df['label'] = df['label'].apply(lambda x: 1 if x >0 else 0)

In [5]:
DF = df.groupby(['Date']).mean().reset_index()
DF.head()

,Date,label
0,2017-11-14,0.500000
1,2017-11-15,0.666667
2,2017-11-16,0.750000
3,2017-11-17,0.500000
4,2017-11-18,0.250000


In [6]:
df.groupby(['Date']).count()

,label,text
Date,,
2017-11-14,6,6
2017-11-15,3,3
2017-11-16,4,4
2017-11-17,8,8
2017-11-18,4,4
...,...,...
2023-05-03,4,4
2023-05-04,6,6
2023-05-05,6,6


In [7]:
DF['label'] = DF['label'].apply(lambda x: 1 if x > 0.5 else 0)

In [8]:
DF

,Date,label
0,2017-11-14,0
1,2017-11-15,1
2,2017-11-16,1
3,2017-11-17,0
4,2017-11-18,0
...,...,...
1980,2023-05-03,0
1981,2023-05-04,0
1982,2023-05-05,0
1983,2023-05-06,0


In [9]:
# Convert date column to datetime object
DF['Date'] = pd.to_datetime(DF['Date'])

# Set date column as index
DF.set_index('Date', inplace=True)

# Check the dataset
print(DF.head())


            label
Date             
2017-11-14      0
2017-11-15      1
2017-11-16      1
2017-11-17      0
2017-11-18      0


In [10]:
# Visualize the dataset
fig = go.Figure()
fig.add_trace(go.Scatter(x=DF.index, y=DF['label'], mode='lines', name='Value'))
fig.update_layout(title='Time Series Visualization')
fig.show()


In [11]:
# Perform first-order differencing
diff = DF.diff().dropna()

# Visualize the differenced dataset
fig = go.Figure()
fig.add_trace(go.Scatter(x=diff.index, y=diff['label'], mode='lines', name='Value'))
fig.update_layout(title='Differenced Time Series Visualization')
fig.show()


In [12]:
# Split the dataset into training and testing sets
train_size = int(len(DF) * 0.8)
train, test = DF[:train_size], DF[train_size:]

# Fit the ARIMA model
model = ARIMA(DF['label'], order=(1,1,1))
model_fit = model.fit()
print(model_fit.summary())


c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



                               SARIMAX Results                                
Dep. Variable:                  label   No. Observations:                 1985
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -1411.603
Date:                Mon, 08 May 2023   AIC                           2829.207
Time:                        23:59:23   BIC                           2845.985
Sample:                             0   HQIC                          2835.370
                               - 1985                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0068      0.022      0.304      0.761      -0.037       0.051
ma.L1         -1.0000      0.191     -5.229      0.000      -1.375      -0.625
sigma2         0.2420      0.094      2.587      0.0

In [13]:
# Evaluate the model using MSE
predictions = []
from tqdm import tqdm
for i in range(len(test)):
    # Forecast the next value
    yhat = model_fit.forecast()
    predictions.append(yhat)
    # Add the actual value to the training set
    obs = test['label']
    train = train.append(pd.DataFrame({'Value': [obs]}, index=[test.index[i]]))
    # Refit the model
    model_fit = ARIMA(train['label'], order=(1,1,1)).fit()
mse = mean_squared_error(test['label'], predictions)
print('MSE:', mse)


c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored whe

MSE: 0.24648986735842618


In [16]:
DF.to_csv('Depression_processed.csv')

In [14]:

# Import required libraries
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA



# Generate forecasts for the next 7 days, 30 days, and 90 days
forecast_1_week = model_fit.forecast(steps=7)
forecast_1_month = model_fit.forecast(steps=30)
forecast_3_months = model_fit.forecast(steps=90)

# Save the forecasts to CSV files
forecast_1_week.to_csv('forecast_1_week.csv',index = False)
forecast_1_month.to_csv('forecast_1_month.csv',index = False)
forecast_3_months.to_csv('forecast_3_months.csv',index = False)

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\tathagat\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

